# 1 references 


-https://huggingface.co/blog/how-to-train-sentence-transformers

-https://github.com/UKPLab/sentence-transformers/tree/master/examples/training/ms_marco

-https://colab.research.google.com/github/UKPLab/sentence-transformers/blob/master/examples/applications/retrieve_rerank/retrieve_rerank_simple_wikipedia.ipynb#scrollTo=2J0Zxgw0artg


# 2 make negatives (after train - recreate and retrain again !)
### - InputExample(texts=[query_text, pos_text, neg_text])
### - InputExample(texts=[row['topic'], row['content']], label=row['label'])

In [1]:

from sentence_transformers import SentenceTransformer, LoggingHandler, util, models, evaluation, losses, InputExample
import pandas as pd
import os
from tqdm import tqdm
import torch

#bi_encoder = SentenceTransformer('all-MiniLM-L6-v2')
bi_encoder = SentenceTransformer('bi-final-0312')

bi_encoder.max_seq_length = 128 
top_k = 50 # 

BASE_DIR = '/kaggle/input/learning-equality-curriculum-recommendations'
BASE_DIR = 'C:/learning'              # временно, для отладки

print ('data reading')
content_data = pd.read_csv(os.path.join(BASE_DIR,'content.csv'))
correlations_data = pd.read_csv(os.path.join(BASE_DIR,'correlations.csv'))
topics_data = pd.read_csv(os.path.join(BASE_DIR,'topics.csv'))
sub_data = pd.read_csv(os.path.join(BASE_DIR,'sample_submission.csv'))

#RFC0220 - only with content
topics_data = topics_data.query('has_content == True').reset_index(drop=True)

#RFC0220 - new tokens and context
topics_data['level'] = topics_data['level'].astype(str)

#topics_data['merged_text'] = topics_data['title'].fillna(' ')
#content_data['merged_text'] =  content_data['title'].fillna(' ')

#topics_data['merged_text'] = topics_data['title'].fillna(' ') + ' [SEP] ' + topics_data['description'].fillna(' ')
#content_data['merged_text'] =  content_data['title'].fillna(' ') + ' [SEP] ' + content_data['description'].fillna(' ')


parentmap = topics_data.set_index('id')['title'].squeeze()
topics_data['parent_title'] = topics_data['parent'].map(parentmap)
topics_data['merged_text'] = topics_data['title'].fillna(' ').str.lower() + ' [SEP] ' + topics_data['description'].fillna(' ').str.lower() \
    +  ' [SEP] ' + topics_data['parent_title'].fillna(' ').str.lower()
content_data['merged_text'] =  content_data['title'].fillna(' ').str.lower() + ' [SEP] ' + content_data['description'].fillna(' ').str.lower()\
    +  ' [SEP] ' + content_data['text'].fillna(' ').str.lower().str[:100]


topic_embeddings = bi_encoder.encode(topics_data['merged_text'], convert_to_tensor=True, show_progress_bar = True)  # 2 минуты
content_embeddings = bi_encoder.encode(content_data['merged_text'], convert_to_tensor=True, show_progress_bar = True)    # 1 минуты


def search_cos_sim(query):
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True, show_progress_bar = False)
    cos_scores = util.cos_sim(query_embedding, content_embeddings)[0]
    hits = torch.topk(cos_scores, k=top_k)
    return (hits[1].cpu().tolist())


def search_cos_sim2(query):
    query_embedding = bi_encoder.encode(query, convert_to_tensor=True, show_progress_bar = False)
    cos_scores = util.cos_sim(query_embedding, content_embeddings)[0]
    hits_t = torch.topk(cos_scores, k=top_k)
    hits = hits_t[1].cpu().tolist()
    scores = hits_t[0].cpu().tolist()
    # давай понизим шумность датасета - иначе он противоречит сам себе - уберем слишком близкие совпадения из обучения
    real_hits=[]
    real_scores=[]
    for idx, element in enumerate(hits):
        if scores[idx]<=0.99:
            real_hits.append(element)
            real_scores.append(scores[idx])
    return (real_hits, real_scores)

# negative examples
t_list=[]
c_list=[]
idx_list =[]
sc_list=[]
for t_idx, row in tqdm(topics_data.iterrows(), total=topics_data.shape[0], position=0, leave=True):
    hits,scores = search_cos_sim2 (row['merged_text'])
    counter_neg = 0
    for hits_idx, hit in enumerate(hits):
        idx_list.append(row['id'])   # здесь счетчик НЕ добавляем в список -- ДОБАВИМ ПОЗЖЕ
        t_list.append(row['merged_text'])
        c_list.append(content_data.iloc[hit]['merged_text'])  #  hit or hit['corpus_id'] is the index in content_data 
        sc_list.append(scores[hits_idx])
        counter_neg +=1
        
# positive examples
ti_list=[]
ci_list=[]
idxi_list =[]
sci_list =[]
for idx, row in tqdm(correlations_data.iterrows(), total=correlations_data.shape[0], position=0, leave=True):
    counter_pos = 0
    for word in row['content_ids'].split():
        idxi_list.append(row['topic_id']+"_"+str(counter_pos))  # здесь счетчик СРАЗУ добавляем в список
        ti_list.append(row['topic_id'])
        ci_list.append(word)
        sci_list.append(1)
        counter_pos +=1
  


data reading


Batches:   0%|          | 0/1923 [00:00<?, ?it/s]

Batches:   0%|          | 0/4814 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████| 61517/61517 [00:05<00:00, 11772.92it/s]


In [2]:
# простой cross-join не нужен - нужна стыковка следующего типа 
#df1 = DataFrame({'key':[1,2,3,1,2,3,1,2], 'col1':[1,2,3,4,5,6,7,8]})
#df2 = DataFrame({'key':[1,2,3], 'col2':[10,11,12]})
#merge(df1, df2,on='key')

idx_list2=[]
temp_topic_id =''
 
for t_idx, element in tqdm(topics_data.iterrows(), total=topics_data.shape[0], position=0, leave=True):
    # сколько получилось позитивных примеров для данного топика 
    temp_count_pos = ti_list.count(element['id'])
    # сколько получилось негативных примеров для данного топика
    temp_count_neg = idx_list.count(element['id'])
    temp_counter = 0
    for i in range (temp_count_neg):
        sub_element = element['id'] + "_" + str(temp_counter)
        idx_list2.append(sub_element)
        temp_counter +=1
        if temp_counter >= temp_count_pos:temp_counter =0
   
print(len(idx_list2), len(idx_list), 'should be equal at the end of procedure')
idx_list = idx_list2 

100%|████████████████████████████████████████████████████████████████████████████| 61517/61517 [48:26<00:00, 21.16it/s]

3051747 3051747 should be equal at the end of procedure


In [4]:

print ('data converting')
        
neg_dataset = pd.DataFrame(list(zip(t_list, c_list,idx_list,sc_list)), columns = ['topic', 'content_neg','topic_id_plus','neg_score'])  

pos_dataset = pd.DataFrame(list(zip(ti_list, ci_list, idxi_list,sci_list)), columns = ['topic_id', 'content_id','topic_id_plus','pos_score'])   
pos_dataset = pd.merge(pos_dataset,topics_data, left_on='topic_id', right_on='id', how='left').loc[:,['merged_text','content_id','topic_id_plus','pos_score']].copy()
pos_dataset = pos_dataset.rename({'merged_text': 'topic'}, axis=1)
pos_dataset = pd.merge(pos_dataset,content_data, left_on='content_id', right_on='id', how='left').loc[:,['topic','merged_text','topic_id_plus','pos_score']].copy()
pos_dataset = pos_dataset.rename({'merged_text': 'content_pos'}, axis=1)

# InputExample(texts=[query_text, pos_text, neg_text])   === 0220 - основной для биэнкодера 
#train_dataset = pd.merge(pos_dataset, neg_dataset, on='topic_id_plus')
# простой cross-join не нужен - нужна стыковка особого типа - см предыдущую ячейку 
train_dataset=pd.merge(neg_dataset,pos_dataset,left_on='topic_id_plus', right_on='topic_id_plus', how='left')

train_dataset = train_dataset.rename({'topic_x': 'query'}, axis=1)
train_dataset = train_dataset.rename({'content_pos': 'pos'}, axis=1)
train_dataset = train_dataset.rename({'content_neg': 'neg'}, axis=1)
train_dataset.drop(['topic_y','topic_id_plus','pos_score','neg_score'], axis=1, inplace=True)
train_dataset.to_csv('train_0220.csv', index = False)     #query,pos,neg
pos_dataset.to_csv('train_0220_pos.csv', index = False)
neg_dataset.to_csv('train_0220_neg.csv', index = False)


# InputExample(texts=[row['topic'], row['content']], label=row['label'])  === 0208
pos_dataset = pos_dataset.rename({'content_pos': 'content'}, axis=1)
pos_dataset = pos_dataset.rename({'pos_score': 'label'}, axis=1)
pos_dataset.drop(['topic_id_plus'], axis=1, inplace=True)
neg_dataset = neg_dataset.rename({'content_neg': 'content'}, axis=1)
neg_dataset = neg_dataset.rename({'neg_score': 'label'}, axis=1)
neg_dataset.drop(['topic_id_plus'], axis=1, inplace=True)
pos_dataset = pos_dataset.append(neg_dataset)
pos_dataset.to_csv('train_0208.csv', index = False)     # for x-encoder - not used now
print ('done - 1203')


data converting


C:\Users\Dmitriy\AppData\Local\Temp\ipykernel_7776\3189652152.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pos_dataset = pos_dataset.append(neg_dataset)


done - 1203


# 3 train biencoder

In [2]:


from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, LoggingHandler, util, models, evaluation, losses, InputExample
import pandas as pd
import os
from tqdm import tqdm
import torch
from sentence_transformers.datasets import NoDuplicatesDataLoader



#bi_encoder = SentenceTransformer('bi-final-0313')
bi_encoder = SentenceTransformer('microsoft/deberta-v3-small')
bi_encoder.max_seq_length = 128
top_k = 50


data= pd.read_csv('train_0220.csv')


No sentence-transformers model found with name C:\Users\Dmitriy/.cache\torch\sentence_transformers\microsoft_deberta-v3-small. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\Dmitriy/.cache\torch\sentence_transformers\microsoft_deberta-v3-small were not used when initializing DebertaV2Model: ['mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are

In [3]:
# https://github.com/UKPLab/sentence-transformers/issues/336
import logging
import os
import csv
import numpy as np
from typing import List, Union
import math
from tqdm.autonotebook import trange

import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from sentence_transformers.evaluation import SentenceEvaluator
from sentence_transformers import InputExample

logger = logging.getLogger(__name__)


class LossEvaluator(SentenceEvaluator):

    def __init__(self, loader, loss_model: nn.Module = None, name: str = '', log_dir: str = None,
                 show_progress_bar: bool = True, write_csv: bool = True):

        # move model to gpu
        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        loss_model.to(self.device)

        self.loader = loader
        self.write_csv = write_csv
        self.logs_writer = SummaryWriter(log_dir=log_dir)
        self.name = name
        self.loss_model = loss_model
        if show_progress_bar is None:
            show_progress_bar = (
                    logger.getEffectiveLevel() == logging.INFO or logger.getEffectiveLevel() == logging.DEBUG)
        self.show_progress_bar = show_progress_bar

        self.csv_file = "loss_evaluation" + ("_" + name if name else '') + "_results.csv"
        self.csv_headers = ["epoch", "steps", "loss"]

    def __call__(self, model, output_path: str = None, epoch: int = -1, steps: int = -1) -> float:

        self.loss_model.eval()

        loss_value = 0
        self.loader.collate_fn = model.smart_batching_collate
        num_batches = len(self.loader)
        data_iterator = iter(self.loader)

               
        with torch.no_grad():
            for _ in trange(num_batches, desc="Iteration", smoothing=0.05, disable=not self.show_progress_bar):
                sentence_features, labels = next(data_iterator)
                # move data to gpu
                for i in range(0, len(sentence_features)):
                    for key, value in sentence_features[i].items():
                        sentence_features[i][key] = sentence_features[i][key].to(self.device)
                labels = labels.to(self.device)
                loss_value += self.loss_model(sentence_features, labels).item()

        final_loss = loss_value / num_batches
        print(loss_value )
        if output_path is not None and self.write_csv:

            csv_path = os.path.join(output_path, self.csv_file)
            output_file_exists = os.path.isfile(csv_path)

            with open(csv_path, newline='', mode="a" if output_file_exists else 'w', encoding="utf-8") as f:
                writer = csv.writer(f)
                if not output_file_exists:
                    writer.writerow(self.csv_headers)

                writer.writerow([epoch, steps, final_loss])

            # ...log the running loss
            self.logs_writer.add_scalar('val_loss',
                                        final_loss,
                                        steps)

        self.loss_model.zero_grad()
        self.loss_model.train()

        return final_loss

In [ ]:
# k-fold validation

from sklearn.model_selection import KFold
kfold = KFold(3, shuffle = True, random_state = 42)

num_epochs = 1

lr = 2e-5
model_save_path = "bi-0314"
#train_loss = losses.ContrastiveLoss(model=bi_encoder)
train_loss = losses.MultipleNegativesRankingLoss(model=bi_encoder)


for train_dataset_n, dev_dataset_n in kfold.split(data):

    train_dataset = data.iloc[train_dataset_n]
    dev_dataset = data.iloc[dev_dataset_n].sample(frac=0.1)

    train_examples = []
    for idx, row in tqdm(train_dataset.iterrows(), total=train_dataset.shape[0], position=0, leave=True):
        #train_examples.append(InputExample(texts=[row['topic'], row['content']], label=row['label']))
        train_examples.append(InputExample(texts=[row['query'], row['pos'], row['neg']]))
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=8)  # 120

    dev_examples = []
    for idx, row in tqdm(dev_dataset.iterrows(), total=dev_dataset.shape[0], position=0, leave=True):
        #dev_examples.append(InputExample(texts=[row['topic'], row['content']], label=row['label']))
        dev_examples.append(InputExample(texts=[row['query'], row['pos'], row['neg']]))
    dev_loader = DataLoader(dev_examples,shuffle=True, batch_size=8)  #110
    dev_evaluator = LossEvaluator(dev_loader, loss_model=train_loss, show_progress_bar= True,log_dir='logs/', name='dev')

    warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

    bi_encoder.fit(train_objectives=[(train_dataloader, train_loss)],
              epochs=num_epochs,
              evaluator=dev_evaluator,
              evaluation_steps=5000,     
              warmup_steps=warmup_steps,
              use_amp=True,
              optimizer_params = {'lr': lr},
              checkpoint_path=model_save_path,
              checkpoint_save_steps=len(train_dataloader),
              show_progress_bar=True
              ) 

bi_encoder.save("bi-final-0314")

100%|███████████████████████████████████████████████████████████████████████| 101725/101725 [00:05<00:00, 19052.05it/s]


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/254313 [00:00<?, ?it/s]

Iteration:   0%|          | 0/12716 [00:00<?, ?it/s]

12614.933639228344


...

4397.7673950672615


Iteration:   0%|          | 0/12716 [00:00<?, ?it/s]

4294.817836695904


Iteration:   0%|          | 0/12716 [00:00<?, ?it/s]

4273.834538573166


In [ ]:
# 1 fold training 

#bi_encoder = SentenceTransformer('bi-final-0308')
#bi_encoder.max_seq_length = 128 

train_dataset = data
dev_dataset = data.sample(frac=0.05)


train_examples = []
for idx, row in tqdm(train_dataset.iterrows(), total=train_dataset.shape[0], position=0, leave=True):
    #train_examples.append(InputExample(texts=[row['topic'], row['content']], label=row['label']))
    train_examples.append(InputExample(texts=[row['query'], row['pos'], row['neg']]))
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=120)  # 120

dev_examples = []
for idx, row in tqdm(dev_dataset.iterrows(), total=dev_dataset.shape[0], position=0, leave=True):
    #dev_examples.append(InputExample(texts=[row['topic'], row['content']], label=row['label']))
    dev_examples.append(InputExample(texts=[row['query'], row['pos'], row['neg']]))
dev_loader = DataLoader(dev_examples,shuffle=True, batch_size=110)  #110
dev_evaluator = LossEvaluator(dev_loader, loss_model=train_loss, show_progress_bar= True,log_dir='logs/', name='dev')

warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

bi_encoder.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          evaluator=dev_evaluator,
          evaluation_steps=5000,     
          warmup_steps=warmup_steps,
          use_amp=True,
          optimizer_params = {'lr': lr},
          checkpoint_path=model_save_path,
          checkpoint_save_steps=len(train_dataloader),
          show_progress_bar=True
          ) 

bi_encoder.save("bi-final-0313b")



In [4]:
bi_encoder.save("bi-final-0313b")

# backup

In [ ]:

print ('data converting')
        
neg_dataset = pd.DataFrame(list(zip(t_list, c_list,idx_list,sc_list)), columns = ['topic', 'content_neg','topic_id_plus','neg_score'])  

pos_dataset = pd.DataFrame(list(zip(ti_list, ci_list, idxi_list,sci_list)), columns = ['topic_id', 'content_id','topic_id_plus','pos_score'])   
pos_dataset = pd.merge(pos_dataset,topics_data, left_on='topic_id', right_on='id', how='left').loc[:,['merged_text','content_id','topic_id_plus','pos_score']].copy()
pos_dataset = pos_dataset.rename({'merged_text': 'topic'}, axis=1)
pos_dataset = pd.merge(pos_dataset,content_data, left_on='content_id', right_on='id', how='left').loc[:,['topic','merged_text','topic_id_plus','pos_score']].copy()
pos_dataset = pos_dataset.rename({'merged_text': 'content_pos'}, axis=1)

# InputExample(texts=[query_text, pos_text, neg_text])   === 0220 - основной для биэнкодера 
#RFC 0220 - cross join
#train_dataset = pd.merge(pos_dataset, neg_dataset, on='topic_id_plus')
#train_dataset=pd.merge(neg_dataset,pos_dataset,left_on='topic_id_plus', right_on='topic_id_plus', how='left')
# кросс пересечение на 50 - получилось записей 13823680
# НЕ кросс пересечение на 100 - получилось записей 6115865

train_que=[]
train_pos=[]
train_neg=[]
temp_counter = 0 

for idx, row in tqdm(neg_dataset.iterrows(), total=neg_dataset.shape[0], position=0, leave=True):
    temp_topic = row['topic_id_plus']
    train_que.append(row['topic'])
    train_neg.append(row['content_neg'])
    temp_pos_dataset = pos_dataset[pos_dataset['topic_id_plus'] == temp_topic].copy().reset_index()
    temp_len_pos= len(temp_pos_dataset)
    if temp_counter >= temp_len_pos: temp_counter = 0
    train_pos.append(temp_pos_dataset.iloc[temp_counter]['content_pos'])
    temp_counter +=1
train_dataset = pd.DataFrame(list(zip(train_que, train_pos,train_neg)), columns = ['query', 'neg','pos'])  

# temp block!!!

train_dataset.head()
